In [2]:
import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"
print(os.getcwd())

c:\Users\utcpret\Documents\NF26\nf26_project


In [3]:
!pip install geopy


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\utcpret\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [18]:
# Initialize regular pandas and other basic libraries
import pandas as pd
import numpy as np
from datetime import datetime, date

# Initialize Spark session
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql import functions as F
from pyspark.sql.types import *
import pyspark.pandas  as ps

# Create SparkSession first
spark = SparkSession.builder \
    .appName("NF26_Project") \
    .getOrCreate()
spark.conf.set("spark.sql.session.timeZone", "Europe/Paris")

# Check PySpark version
print(f"PySpark version: {spark.version}")

# Import ETL functions - these don't depend on pyspark.pandas
from etl_warehouse import *

# Define a helper function to read CSV with pandas and convert to Spark DataFrame
def read_csv_to_spark(file_path, sep=';', encoding='utf-8', index_col=None):
    """Read CSV with pandas and convert to Spark DataFrame"""
    pdf = pd.read_csv(file_path, sep=sep, encoding=encoding)
    if index_col:
        pdf.set_index(index_col, inplace=True)
    return spark.createDataFrame(pdf)

PySpark version: 3.5.5


In [10]:
spark = SparkSession.builder \
    .appName("NF26_Project") \
    .getOrCreate()
spark.conf.set("spark.sql.session.timeZone", "Europe/Paris")

In [11]:
## Définition du fuseau horaire sur paris ##

spark.conf.set("spark.sql.session.timeZone", "Europe/Paris")


EXECUTION de l'ETL boucle avec tous les jours de 2024.

In [7]:
process_full_year()

Starting ETL for all 366 days in 2024...

Processing date 20240101 (1/366)
Démarrage de l'ETL pour la date: 20240101
Erreur lors de la création/mise à jour de la table MISSION: Aucune donnée trouvée pour MISSION à la date 20240101
Traceback (most recent call last):
  File "c:\Users\utcpret\Documents\NF26\nf26_project\etl_warehouse.py", line 222, in creer_table_fait_mission
    df_missions = lire_fichiers_source(date_str, "MISSION")
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\utcpret\Documents\NF26\nf26_project\etl_warehouse.py", line 78, in lire_fichiers_source
    raise FileNotFoundError(f"Aucune donnée trouvée pour {type_donnees} à la date {date_str}")
FileNotFoundError: Aucune donnée trouvée pour MISSION à la date 20240101

Erreur lors de la création/mise à jour de la table MATERIEL: Aucune donnée trouvée pour MATERIEL_INFORMATIQUE à la date 20240101
Traceback (most recent call last):
  File "c:\Users\utcpret\Documents\NF26\nf26_project\etl_warehouse

TABLE PERSONNEL
pas besoin de la créer de manière journalière car on pouvait l'extraire directement

In [44]:
#BERLIN
psdf_personnel_BERLIN = ps.read_csv('BDD_BGES/BDD_BGES/BDD_BGES_BERLIN/PERSONNEL_BERLIN.txt', index_col='ID_PERSONNEL', sep=';', encoding='utf-8')
sdf_personnel_BERLIN = psdf_personnel_BERLIN.to_spark(index_col='ID_PERSONNEL')

#LONDON
psdf_personnel_LONDON = ps.read_csv('BDD_BGES/BDD_BGES/BDD_BGES_LONDON/PERSONNEL_LONDON.txt', index_col='ID_PERSONNEL', sep=';', encoding='utf-8')
sdf_personnel_LONDON = psdf_personnel_LONDON.to_spark(index_col='ID_PERSONNEL')

#LOSANGELES
psdf_personnel_LOSANGELES = ps.read_csv('BDD_BGES/BDD_BGES/BDD_BGES_LOSANGELES/PERSONNEL_LOSANGELES.txt', index_col='ID_PERSONNEL', sep=';', encoding='utf-8')
sdf_personnel_LOSANGELES = psdf_personnel_LOSANGELES.to_spark(index_col='ID_PERSONNEL')

#NEWYORK
psdf_personnel_NEWYORK = ps.read_csv('BDD_BGES/BDD_BGES/BDD_BGES_NEWYORK/PERSONNEL_NEWYORK.txt', index_col='ID_PERSONNEL', sep=';', encoding='utf-8')
sdf_personnel_NEWYORK = psdf_personnel_NEWYORK.to_spark(index_col='ID_PERSONNEL')

#PARIS
psdf_personnel_PARIS = ps.read_csv('BDD_BGES/BDD_BGES/BDD_BGES_PARIS/PERSONNEL_PARIS.txt', index_col='ID_PERSONNEL', sep=';', encoding='utf-8')
sdf_personnel_PARIS = psdf_personnel_PARIS.to_spark(index_col='ID_PERSONNEL')

#SHANGHAI
psdf_personnel_SHANGHAI = ps.read_csv('BDD_BGES/BDD_BGES/BDD_BGES_SHANGHAI/PERSONNEL_SHANGHAI.txt', index_col='ID_PERSONNEL', sep=';', encoding='utf-8')
sdf_personnel_SHANGHAI = psdf_personnel_SHANGHAI.to_spark(index_col='ID_PERSONNEL')
 
 
 
# Code corrigé pour la dimension personnel
import os
from pyspark.sql.functions import lit, col

# Créer le répertoire DIM_PERSONNEL s'il n'existe pas
os.makedirs("DATA_WAREHOUSE/DIM_PERSONNEL", exist_ok=True)

# Ajouter la colonne VILLE pour indiquer la provenance
sdf_personnel_berlin_with_city = sdf_personnel_BERLIN.withColumn("VILLE", lit("BERLIN"))
sdf_personnel_london_with_city = sdf_personnel_LONDON.withColumn("VILLE", lit("LONDON"))
sdf_personnel_la_with_city = sdf_personnel_LOSANGELES.withColumn("VILLE", lit("LOSANGELES"))
sdf_personnel_ny_with_city = sdf_personnel_NEWYORK.withColumn("VILLE", lit("NEWYORK"))
sdf_personnel_paris_with_city = sdf_personnel_PARIS.withColumn("VILLE", lit("PARIS"))
sdf_personnel_shanghai_with_city = sdf_personnel_SHANGHAI.withColumn("VILLE", lit("SHANGHAI"))

# Union de tous les DataFrames avec la colonne VILLE
sdf_personnel_all_with_city = sdf_personnel_berlin_with_city.union(sdf_personnel_london_with_city) \
    .union(sdf_personnel_la_with_city).union(sdf_personnel_ny_with_city) \
    .union(sdf_personnel_paris_with_city).union(sdf_personnel_shanghai_with_city)

sdf_personnel_all_with_city.select("FONCTION_PERSONNEL").distinct().show(truncate=False, n = 1000)
mapping_fonctions = {
    "Ökonom": "Economiste",
    "Führungskraft": "Cadre",
    "Personalleiter": "DRH",
    "Computeringenieur": "Ingénieur Informaticien",
    "Dateningenieur": "Ingénieur Data",
    "Computer Engineer": "Ingénieur Informaticien",
    "HRD": "DRH",
    "Business Executive": "Cadre",
    "Economist": "Economiste",
    "Data Engineer": "Ingénieur Data",
    "Ingénieur Informaticien": "Ingénieur Informaticien",
    "Ingénieur Data": "Ingénieur Data",
    "Economiste": "Economiste",
    "DRH": "DRH",
    "Cadre": "Cadre"
}

sdf_personnel_all_with_city = sdf_personnel_all_with_city.toPandas()

sdf_personnel_all_with_city['FONCTION_PERSONNEL'] = sdf_personnel_all_with_city['FONCTION_PERSONNEL'].map(mapping_fonctions)

sdf_personnel_all_with_city.to_csv('DATA_WAREHOUSE/DIM_PERSONNEL/dimension_personnel.csv', sep=';', index=False, encoding='utf-8')
# Display the unique homogenized values (optional)
print(sdf_personnel_all_with_city['FONCTION_PERSONNEL'].unique())

+-----------------------+
|FONCTION_PERSONNEL     |
+-----------------------+
|Ökonom                 |
|Führungskraft          |
|Personalleiter         |
|Computeringenieur      |
|Dateningenieur         |
|Computer Engineer      |
|HRD                    |
|Business Executive     |
|Economist              |
|Data Engineer          |
|Ingénieur Informaticien|
|Ingénieur Data         |
|Economiste             |
|DRH                    |
|Cadre                  |
+-----------------------+

['Ingénieur Data' 'Cadre' 'Economiste' 'DRH' 'Ingénieur Informaticien']


In [58]:
mission_mapping = {
    "Geschäftstreffen": "Réunion",
    "Konferenz": "Conférence",
    "Schulung": "Formation",
    "Meeting": "Réunion",
    "Entwicklung": "Développement",
    "Business Meeting": "Réunion",
    "Conference": "Conférence",
    "Team Meeting": "Réunion",
    "Development": "Développement",
    "Vocational Training": "Formation",
    "Conférence": "Conférence",
    "Formation": "Formation",
    "Réunion": "Réunion",
    "Rencontre entreprises": "Réunion",
    "Développement": "Développement"
}

# Corrigez la lecture du fichier en utilisant la virgule comme séparateur
pandas_df = pd.read_csv('DATA_WAREHOUSE/DIM_MISSION/dimension_mission.csv', sep=',', encoding='utf-8')

# Vérifiez les colonnes après correction
print(pandas_df.columns)

# Maintenant vous pouvez accéder à la colonne TYPE_MISSION correctement
if 'TYPE_MISSION' in pandas_df.columns:
    pandas_df['TYPE_MISSION'] = pandas_df['TYPE_MISSION'].map(mission_mapping).fillna('Autre')
    
    # Sauvegardez le fichier modifié (conservez le même format de séparateur)
    pandas_df.to_csv('DATA_WAREHOUSE/DIM_MISSION/dimension_mission.csv', sep=',', index=False, encoding='utf-8')
    
    # Rechargez la vue temporaire
    dim_mission = spark.read.csv("DATA_WAREHOUSE/DIM_MISSION/dimension_mission.csv", header=True, inferSchema=True, sep=',')
    dim_mission.createOrReplaceTempView("mission")
    print("Mappings de TYPE_MISSION appliqués avec succès")
else:
    print("La colonne TYPE_MISSION n'existe pas. Voici les colonnes disponibles:", pandas_df.columns)


Index(['ID_MISSION', 'TYPE_MISSION', 'VILLE_DEPART', 'PAYS_DEPART',
       'VILLE_DESTINATION', 'PAYS_DESTINATION', 'TRANSPORT', 'ALLER_RETOUR',
       'GES'],
      dtype='object')
Mappings de TYPE_MISSION appliqués avec succès


Chargement de toutes les tables de dimension et de faits nécessaires pour les analyses

In [59]:
import pyspark.pandas as ps 
from pyspark.sql import functions as F 
from pyspark.sql.types import *

# Lecture du fichier dimension_personnel avec séparateur ";"
dim_personnel = spark.read.csv("DATA_WAREHOUSE/DIM_PERSONNEL/dimension_personnel.csv", 
                             header=True, 
                             inferSchema=True,
                             sep=";")  # Spécifier le séparateur ";"

# Lecture des autres fichiers avec séparateur "," (valeur par défaut)
dim_materiel = spark.read.csv("DATA_WAREHOUSE/DIM_MATERIEL/dimension_materiel.csv", 
                             header=True, 
                             inferSchema=True)
dim_date = spark.read.csv("DATA_WAREHOUSE/DIM_DATE/dimension_date.csv", 
                         header=True, 
                         inferSchema=True) 
dim_mission = spark.read.csv("DATA_WAREHOUSE/DIM_MISSION/dimension_mission.csv", 
                           header=True, 
                           inferSchema=True) 
faits_materiel = spark.read.csv("DATA_WAREHOUSE/FAITS_MATERIEL/faits_materiel.csv", 
                              header=True, 
                              inferSchema=True) 
faits_mission = spark.read.csv("DATA_WAREHOUSE/FAITS_MISSION/faits_mission.csv", 
                             header=True, 
                             inferSchema=True)

# Création des vues temporaires
dim_personnel.createOrReplaceTempView("personnel") 
dim_materiel.createOrReplaceTempView("materiel") 
dim_date.createOrReplaceTempView("dates") 
dim_mission.createOrReplaceTempView("mission") 
faits_materiel.createOrReplaceTempView("faits_materiel") 
faits_mission.createOrReplaceTempView("faits_mission")

print("\nToutes les vues temporaires ont été créées avec succès.")


Toutes les vues temporaires ont été créées avec succès.


Réponse aux questions 1 a 4

In [ ]:
# Question 1: Combien d'ingénieurs informaticiens travaillent sur le site de Paris?
q1 = spark.sql("""
    SELECT COUNT(*) as nb_ingenieurs_info_paris
    FROM personnel
    WHERE `FONCTION_PERSONNEL` = 'Ingénieur Informaticien' 
    AND `VILLE` = 'PARIS'
""")
q1.show()

# Question 2: Combien d'ingénieurs Data travaillent sur les sites de London?
q2 = spark.sql("""
    SELECT COUNT(*) as nb_ingenieurs_data_london
    FROM personnel
    WHERE FONCTION_PERSONNEL = 'Ingénieur Data'
    AND VILLE = 'LONDON'
""")
q2.show()

# Question 3: Combien de cadres travaillent dans l'organisation (tous sites compris)?
q3 = spark.sql("""
    SELECT COUNT(*) as nb_cadres_total
    FROM personnel
    WHERE FONCTION_PERSONNEL = 'Cadre'
""")
q3.show()

q4 = spark.sql("""
    SELECT COUNT(DISTINCT f.ID_MATERIELINFO) as nb_pc_portables
    FROM faits_materiel f
    JOIN materiel m ON f.ID_MATERIELINFO = m.ID_MATERIELINFO
    JOIN dates d ON f.KeyDate = d.KeyDate
    WHERE m.TYPE LIKE '%PC portable%'
    AND d.MOIS BETWEEN 5 AND 10
    AND d.ANNEE = 2024
""")
q4.show()

+------------------------+
|nb_ingenieurs_info_paris|
+------------------------+
|                    1831|
+------------------------+

+-------------------------+
|nb_ingenieurs_data_london|
+-------------------------+
|                     1568|
+-------------------------+

+---------------+
|nb_cadres_total|
+---------------+
|           3955|
+---------------+

+---------------+
|nb_pc_portables|
+---------------+
|           1820|
+---------------+



In [22]:
# Fix Hadoop environment issue for Windows
import os
import sys
import tempfile
import pandas as pd

# Create a temporary directory for Hadoop
hadoop_home = os.path.join(tempfile.gettempdir(), "hadoop")
os.makedirs(os.path.join(hadoop_home, "bin"), exist_ok=True)

# Set Hadoop environment variables
os.environ["HADOOP_HOME"] = hadoop_home
os.environ["hadoop.home.dir"] = hadoop_home
os.environ["PATH"] = os.environ["PATH"] + os.pathsep + os.path.join(hadoop_home, "bin")


# Analyse de l'Impact Carbone

Réponses aux questions d'analyse d'impact carbone avec Spark SQL.

In [ ]:
# Question 5: Quelle a été l'impact carbone des PC portables entre mai et octobre 2024?
q5 = spark.sql("""
    SELECT SUM(m.GES) as impact_carbone_pc_portables
    FROM materiel m
    JOIN faits_materiel f ON m.ID_MATERIELINFO = f.ID_MATERIELINFO
    JOIN dates d ON f.KeyDate = d.KeyDate
    WHERE m.TYPE LIKE '%PC portable%'
      AND d.MOIS BETWEEN 5 AND 10
      AND d.ANNEE = 2024
""")
print("Question 5: Impact carbone des PC portables (mai-octobre 2024)")
q5.show()

# Question 6: Quelle a été l'impact carbone des PC fixes sans écran achetés par les ingénieurs Data 
# entre mai et septembre 2024 sur les sites de Paris et New-York?
q6 = spark.sql("""
    SELECT SUM(m.GES) as impact_carbone_pc_fixes
    FROM materiel m
    JOIN faits_materiel f ON m.ID_MATERIELINFO = f.ID_MATERIELINFO
    JOIN dates d ON f.KeyDate = d.KeyDate
    JOIN personnel p ON f.ID_PERSONNEL = p.ID_PERSONNEL
    WHERE m.TYPE LIKE '%PC fixe sans ecran%'
      AND p.FONCTION_PERSONNEL = 'Ingénieur Data'
      AND (p.VILLE = 'PARIS' OR p.VILLE = 'NEWYORK')
      AND d.MOIS BETWEEN 5 AND 9
      AND d.ANNEE = 2024
""")
print("Question 6: Impact carbone des PC fixes sans écran (ingénieurs Data, Paris et NY, mai-septembre 2024)")
q6.show()

# Question 7: Quelle a été l'impact carbone des Ecrans achetés par les cadres 
# entre mai et septembre 2024 sur tous les sites de l'organisation?
q7 = spark.sql("""
    SELECT SUM(m.GES) as impact_carbone_ecrans
    FROM materiel m
    JOIN faits_materiel f ON m.ID_MATERIELINFO = f.ID_MATERIELINFO
    JOIN dates d ON f.KeyDate = d.KeyDate
    JOIN personnel p ON f.ID_PERSONNEL = p.ID_PERSONNEL
    WHERE m.TYPE LIKE '%Ecran%'
      AND p.FONCTION_PERSONNEL = 'Cadre'
      AND d.MOIS BETWEEN 5 AND 9
      AND d.ANNEE = 2024
""")
print("Question 7: Impact carbone des écrans achetés par les cadres (mai-septembre 2024)")
q7.show()

# Question 8: Quelle a été l'impact carbone des missions sur les sites Européens
# en juillet 2024?
q8 = spark.sql("""
    SELECT SUM(m.GES) as impact_carbone_missions_europe
    FROM mission m
    JOIN faits_mission f ON m.ID_MISSION = f.ID_MISSION
    JOIN dates d ON f.KeyDate = d.KeyDate
    WHERE (m.PAYS_DEPART IN ('France', 'Allemagne', 'England') 
           OR m.PAYS_DESTINATION IN ('France', 'Allemagne', 'England'))
      AND d.MOIS = 7
      AND d.ANNEE = 2024
""")
print("Question 8: Impact carbone des missions sur les sites européens (juillet 2024)")
q8.show()

# Question 9: Quels ont été les 5 jours les plus impactants concernant les missions 
# en avion sur l'ensemble des sites de l'organisation?
q9 = spark.sql("""
    SELECT d.DATE_PARIS, SUM(m.GES) as impact_total
    FROM mission m
    JOIN faits_mission f ON m.ID_MISSION = f.ID_MISSION
    JOIN dates d ON f.KeyDate = d.KeyDate
    WHERE m.TRANSPORT = 'Avion'
    GROUP BY d.DATE_PARIS
    ORDER BY impact_total DESC
    LIMIT 5
""")
print("Question 9: Les 5 jours les plus impactants pour les missions en avion")
q9.show()

# Question 10: Quelle a été la fonction du personnel qui a eu le plus d'impact concernant 
# les missions et le matériel informatique sur l'ensemble des sites de l'organisation?
q10 = spark.sql("""
    SELECT p.FONCTION_PERSONNEL, 
           SUM(COALESCE(mis.GES, 0) + COALESCE(mat.GES, 0)) as impact_total
    FROM personnel p
    LEFT JOIN faits_mission fm ON p.ID_PERSONNEL = fm.ID_PERSONNEL
    LEFT JOIN mission mis ON fm.ID_MISSION = mis.ID_MISSION
    LEFT JOIN faits_materiel fmat ON p.ID_PERSONNEL = fmat.ID_PERSONNEL
    LEFT JOIN materiel mat ON fmat.ID_MATERIELINFO = mat.ID_MATERIELINFO
    GROUP BY p.FONCTION_PERSONNEL
    ORDER BY impact_total DESC
    LIMIT 1
""")
print("Question 10: Fonction du personnel avec le plus d'impact (missions et matériel)")
q10.show()

# Question 11: Quel site a eu le plus d'impact concernant les missions et le 
# matériel informatique sur l'ensemble des sites de l'organisation?
q11 = spark.sql("""
    SELECT p.VILLE, 
           SUM(COALESCE(mis.GES, 0) + COALESCE(mat.GES, 0)) as impact_total
    FROM personnel p
    LEFT JOIN faits_mission fm ON p.ID_PERSONNEL = fm.ID_PERSONNEL
    LEFT JOIN mission mis ON fm.ID_MISSION = mis.ID_MISSION
    LEFT JOIN faits_materiel fmat ON p.ID_PERSONNEL = fmat.ID_PERSONNEL
    LEFT JOIN materiel mat ON fmat.ID_MATERIELINFO = mat.ID_MATERIELINFO
    GROUP BY p.VILLE
    ORDER BY impact_total DESC
    LIMIT 1
""")
print("Question 11: Site avec le plus d'impact (missions et matériel)")
q11.show()

# Question 12: Quel a été l'impact carbone des missions reliant chaque site
# durant le mois de mai 2024?
q12 = spark.sql("""
    SELECT mis.VILLE_DEPART, mis.VILLE_DESTINATION, SUM(mis.GES) as impact_carbone
    FROM mission mis
    JOIN faits_mission fm ON mis.ID_MISSION = fm.ID_MISSION
    JOIN dates d ON fm.KeyDate = d.KeyDate
    WHERE mis.VILLE_DEPART IN ('PARIS', 'BERLIN', 'LONDON', 'LOSANGELES', 'NEWYORK', 'SHANGHAI')
      AND mis.VILLE_DESTINATION IN ('PARIS', 'BERLIN', 'LONDON', 'LOSANGELES', 'NEWYORK', 'SHANGHAI')
      AND d.MOIS = 5
      AND d.ANNEE = 2024
    GROUP BY mis.VILLE_DEPART, mis.VILLE_DESTINATION
    ORDER BY impact_carbone DESC
""")
print("Question 12: Impact carbone des missions entre sites (mai 2024)")
q12.show(20)  # Show more rows to see all site combinations

# Question 13: Quel a été l'impact carbone des séminaires en juillet 2024 pour 
# les employés de Los Angeles?
q13 = spark.sql("""
    SELECT SUM(mis.GES) as impact_seminaires
    FROM mission mis
    JOIN faits_mission fm ON mis.ID_MISSION = fm.ID_MISSION
    JOIN dates d ON fm.KeyDate = d.KeyDate
    JOIN personnel p ON fm.ID_PERSONNEL = p.ID_PERSONNEL
    WHERE mis.TYPE_MISSION = 'séminaire'
      AND p.VILLE = 'LOSANGELES'
      AND d.MOIS = 7
      AND d.ANNEE = 2024
""")
print("Question 13: Impact carbone des séminaires (employés de Los Angeles, juillet 2024)")
q13.show()

# Question 14: Quel a secteur d'activité a été le plus impactant pour les missions
# "conférences" entre mai et octobre 2024?
q14 = spark.sql("""
    SELECT p.FONCTION_PERSONNEL, SUM(mis.GES) as impact_total
    FROM mission mis
    JOIN faits_mission fm ON mis.ID_MISSION = fm.ID_MISSION
    JOIN dates d ON fm.KeyDate = d.KeyDate
    JOIN personnel p ON fm.ID_PERSONNEL = p.ID_PERSONNEL
    WHERE mis.TYPE_MISSION = 'Conférence'
      AND d.MOIS BETWEEN 5 AND 10
      AND d.ANNEE = 2024
    GROUP BY p.FONCTION_PERSONNEL
    ORDER BY impact_total DESC
    LIMIT 1
""")
print("Question 14: Secteur d'activité le plus impactant pour les conférences (mai-octobre 2024)")
q14.show()

# Question 15: Quel a été l'âge moyen des employés qui sont partis en formations
# entre juillet et septembre 2024?
q15 = spark.sql("""
    SELECT AVG(YEAR(CURRENT_DATE()) - YEAR(p.DATE_NAISSANCE)) as age_moyen
    FROM personnel p
    JOIN faits_mission fm ON p.ID_PERSONNEL = fm.ID_PERSONNEL
    JOIN mission mis ON fm.ID_MISSION = mis.ID_MISSION
    JOIN dates d ON fm.KeyDate = d.KeyDate
    WHERE mis.TYPE_MISSION = 'Formation'
      AND d.MOIS BETWEEN 7 AND 9
      AND d.ANNEE = 2024
""")
print("Question 15: Âge moyen des employés partis en formation (juillet-septembre 2024)")
q15.show()

# Question 16: Quelle destination a été la plus impactante (en cumul) entre mai et octobre 2024?
q16 = spark.sql("""
    SELECT mis.VILLE_DESTINATION, SUM(mis.GES) as impact_total
    FROM mission mis
    JOIN faits_mission fm ON mis.ID_MISSION = fm.ID_MISSION
    JOIN dates d ON fm.KeyDate = d.KeyDate
    WHERE d.MOIS BETWEEN 5 AND 10
      AND d.ANNEE = 2024
    GROUP BY mis.VILLE_DESTINATION
    ORDER BY impact_total DESC
    LIMIT 1
""")
print("Question 16: Destination la plus impactante (mai-octobre 2024)")
q16.show()

Question 5: Impact carbone des PC portables (mai-octobre 2024)
+---------------------------+
|impact_carbone_pc_portables|
+---------------------------+
|         1102.9879999999798|
+---------------------------+

Question 6: Impact carbone des PC fixes sans écran (ingénieurs Data, Paris et NY, mai-septembre 2024)
+-----------------------+
|impact_carbone_pc_fixes|
+-----------------------+
|     152.64999999999986|
+-----------------------+

Question 7: Impact carbone des écrans achetés par les cadres (mai-septembre 2024)
+---------------------+
|impact_carbone_ecrans|
+---------------------+
|    44.80000000000003|
+---------------------+

Question 8: Impact carbone des missions sur les sites européens (juillet 2024)
+------------------------------+
|impact_carbone_missions_europe|
+------------------------------+
|             8543.530308454261|
+------------------------------+

Question 9: Les 5 jours les plus impactants pour les missions en avion
+----------+------------------+
|D